# Homework 3

## FINM 37400 - 2025

### UChicago Financial Mathematics

### HW Group B 18
* Matheus Raka Pradnyatama
* Jacob Simeral

In [38]:
import pandas as pd
import numpy as np
import datetime
import holidays
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sklearn.linear_model import LinearRegression

from scipy.optimize import minimize
from scipy import interpolate

from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
# The code here is written with the help of OpenAI's ChatGPT.

In [39]:
def price_treasury_ytm(yield_to_maturity, coupon_rate, face_value, time_to_maturity, payments_per_year=2):
    """
    Prices a Treasury bond based on its yield to maturity, coupon rate, face value, and time to maturity.

    Parameters:
        yield_to_maturity (float): The annual yield to maturity (as a decimal, e.g., 0.03 for 3%).
        coupon_rate (float): The annual coupon rate (as a decimal, e.g., 0.02 for 2%).
        face_value (float): The face value (par value) of the bond.
        time_to_maturity (float): The time to maturity of the bond in years.
        payments_per_year (int): The number of coupon payments per year (default is 2 for semi-annual payments).

    Returns:
        float: The price of the Treasury bond.
    """
    # Calculate the coupon payment per period
    coupon_payment = coupon_rate * face_value / payments_per_year

    # Total number of periods
    total_periods = int(time_to_maturity * payments_per_year)

    # Periodic yield to maturity
    periodic_yield = yield_to_maturity / payments_per_year

    # Price calculation: Sum of discounted coupon payments + discounted face value
    price = 0

    for t in range(1, total_periods + 1):
        price += coupon_payment / (1 + periodic_yield) ** t

    # Add the present value of the face value (paid at maturity)
    price += face_value / (1 + periodic_yield) ** total_periods

    return price

In [40]:
# Duration for a fixed rate bond
# freq = frequency of compounding in a year
# tau = years-to-maturity

def duration_closed_formula(tau, ytm, coupon_rate=None, freq=2):

    if coupon_rate is None:
        coupon_rate = ytm
        
    # y_tilde = ytm/frequency
    y = ytm/freq
    # c_tilde = coupon rate/ frequency
    c = coupon_rate/freq
    # tau_tilde= tau * frequency
    T = tau * freq
        
    if coupon_rate==ytm:
        duration = (1+y)/y  * (1 - 1/(1+y)**T)
        
    else:
        duration = (1+y)/y - (1+y+T*(c-y)) / (c*((1+y)**T-1)+y)

    duration /= freq
    
    return duration


***

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (C): Spread and Swap Spread in November 2008

## Simplification of the setup

The date is Nov 4, 2008.

**Treasury bond**
* Suppose the Treasury bond matures exactly 30 years later, on Nov 4, 2038 rather than May 15, 2008. 
* The YTM of this freshly issued treasury is 4.193\% with a semiannual coupon of 4.50\%, same as is given in the case. (So we're just changing the maturity date to simplify things, but keeping the market data.)

**Swap**
* The fixed leg of the swap pays semiannually, with swap rate of 4.2560\%, as given in the case.
* The floating leg of the swap also pays semiannually--not quarterly--such that the payment dates are identical on both legs. Thus, it also resets the floating rate semiannually, not quarterly.
* The floating rate of the swap equals the repo rate used in the trade. Thus, these two rates cancel in the financing of the trade. (No need to consider the TED spread.) 

## Case Clarifications


### Duration Quotes
Bond
* Quote: Val01 of bond is .1746 per bp per $1 face value
* Class terminology: Modified dollar duration is .1746 per $100 face value

Swap
* Quote: DV01 of swap is 1.7mm per 1 billion notional.
* Class terminology: Modified dollar duration is 100(1.7/1000) per $100 face value.

Thus, modified dollar duration for each per 100 face is
* Bond = .1746
* Swap = .1700

### Hedge Ratio

In figuring out the hedge ratio, they set up the hedge per dollar of face value. 

    *so Mills would need to buy face amount $0.97 billion*
    
No, this hedge should be for market value, not face amount given that the case is already using **modified** duration which includes the dirty price.
    

### Maturity Mismatch

The maturity of the bond is August 2038, whereas the date is Nov 2008. Thus, the bond has less than 30 years to maturity, yet he is entering a 30-year swap. 

For simplicity, we imagine the bond is issued in Nov 2008 and thus has maturity of 30 years at the time of the case.

However, then the case quotes for the Nov price and YTM of the bond no longer are accurate. Use one and adjust the other. Namely, we could...
    * use the Nov 4 **YTM** quoted in the case, and re-adjust the the bond.
    * use the Nov 4 **price** quoted in the case, and re-adjust the YTM.
    
We do the former, keep the quoted YTM, assume time-to-maturity of `30`, and recalculate the Nov 2008 price. (It is close to the quoted price in the case.)

***

In [41]:
# Case Parameter
ytm = 4.193/100  # Annual yield to maturity
coupon_rate = 4.5/100  # Annual coupon rate
time_to_maturity = 30  # Years to maturity
swap_rate = 4.256/100 # Fixed rate of the swap 

par_value = 100
haircut = 0.02

## 1.0.

Report the price of the 30-year T-bond in Nov 2008. Given the discussion about `Maturity Mismatch`, we are repricing the bond, so it will not exactly equal `105` as reported in the case.

In [75]:
ytm = 4.193/100  # Annual yield to maturity
coupon_rate = 4.5/100  # Annual coupon rate
face_value = 100  # Face value = Par Value
time_to_maturity = 30  # Years to maturity

treasury_price_nov08 = price_treasury_ytm(ytm, coupon_rate, face_value, time_to_maturity)
print(f"The price of the 30-Year Treasury bond is: ${treasury_price_nov08:.2f}")

The price of the 30-Year Treasury bond is: $105.21


## 1.1

List the projected cashflows on May 4, 2009, exactly six months into the trade, on the first coupon and swap date.

#### Clarification
List these cashflows for face value of $1B, not the $0.97B noted in the case. As mentioned in "Case Clarifications", we will not use this number. Rather, we calculate our own hedge ratio in a problem below.

In [43]:
# No need to account for the repo rate or the swap's floating payment, 
# as they both are modeled in this problem with SOFR, and thus net to zero
SOFR = np.nan

In [44]:
face_value = 100 # Unit is tens of millions
coupon_rate = 4.5/100  # 4.5% annual coupon rate
swap_rate = 4.256/100 # 4.256% fixed rate of the swap 

CF = pd.DataFrame(index=['T bond','Repo','Swap (floating leg)','Swap (fixed leg)'],columns=['May 2009'],dtype=float)

CF.loc['Repo'] = -SOFR # Pay SOFR (floating) as repo rate to borrow cash for buying 30Y Treasury

cashflow_treasury = face_value * coupon_rate /2 # Receive fixed rate coupon payment from Treasuries
CF.loc[['T bond']] = cashflow_treasury

CF.loc['Swap (floating leg)'] = SOFR # Receive floating rate from swap

cashflow_swap_fix = -face_value * swap_rate/2 # Pay fixed rate on swap
CF.loc[['Swap (fixed leg)']] = cashflow_swap_fix

CF.loc['Net Payment'] = CF.sum(axis=0)
CF.style.format('${:,.2f}')
# We are receiving more than we are paying

,May 2009
T bond,$2.25
Repo,$nan
Swap (floating leg),$nan
Swap (fixed leg),$-2.13
Net Payment,$0.12


## 1.2

What is the duration of...
* the T-bond
* the swap

Remember that...
* the swap can be decomposed into a fixed-rate bond and a floating-rate note
* a floating-rate note has duration equal to the time until the next reset. Thus, at initialization, it has duration equal to 0.5 years.

Is the duration for the "paying-fixed" swap positive or negative? Is it bigger or smaller in magnitude than the T-bond?

For this problem, calculate the Macauley duration and the dollar (Macauley) duration.

In [77]:
# Treasury Bond
tau = 30  # 30 years-to-maturity
ytm = 4.193/100  # 4.193% annual yield to maturity
coupon_rate = 4.5/100  # 4.5% annual coupon rate
freq = 2       # Semi-annual payments

dur_treasury = duration_closed_formula(tau, ytm, coupon_rate, freq=2)
print(f"Duration of the Treasury Bond: {dur_treasury:.4f}")

# Dollar Duration = Duration x Treasury Price (calculated)
dur_dollar_treasury = dur_treasury * treasury_price_nov08
print(f"Dollar Duration of the Treasury Bond: ${dur_dollar_treasury:,.4f}")

Duration of the Treasury Bond: 17.0836
Dollar Duration of the Treasury Bond: $1,797.4251


In [46]:
# Fixed Leg of Swap
tau = 30  # 30 years-to-maturity
swap_rate = 4.256/100 # 4.256% fixed rate of the swap 
coupon_rate = None
freq = 2       # Semi-annual payments

dur_swap_fix = duration_closed_formula(tau, swap_rate, coupon_rate, freq=2)
print(f"Duration of the Fixed Leg of the Swap: {dur_swap_fix:.4f}")

# Dollar duration = Duration x Price
# Swaps are combinations of bonds and notes
# Price = 100
dur_dollar_swap_fix = dur_swap_fix * 100
print(f"Dollar Duration of the Fixed Leg of the Swap: ${dur_dollar_swap_fix:,.4f}\n")

dur_swap_float = 0.5 # Semi-annual reset
dur_dollar_swap_float = dur_swap_float * 100

print(f"Duration of the Floating Leg of the Swap: {dur_swap_float}")
print(f"Dollar Duration of the Fixed Leg of the Swap: ${dur_dollar_swap_float:,.4f}")

Duration of the Fixed Leg of the Swap: 17.2127
Dollar Duration of the Fixed Leg of the Swap: $1,721.2744

Duration of the Floating Leg of the Swap: 0.5
Dollar Duration of the Fixed Leg of the Swap: $50.0000


In [47]:
dur_repo = 0.5 # Semi-annual reset
# Borrowed Repo = treasury price * (1-haircut)
repo = treasury_price_nov08 * (1 - haircut)
dur_dollar_repo = dur_repo * repo

print(f"Duration of the Repo: {dur_repo}")
print(f"Repo Amount: ${repo:,.4f}")
print(f"Dollar Duration of the Repo: ${dur_dollar_repo:,.4f}")

Duration of the Repo: 0.5
Repo Amount: $98.0000
Dollar Duration of the Repo: $49.0000


In [78]:
tab_net = pd.DataFrame(dtype=float, index=['T repo','swap'], columns=['duration','dollar duration'])
# For T Repo: Treasury Bond (fixed) - Repo (floating)
net_dur_repo = dur_treasury - dur_repo
net_dur_dollar_repo = dur_dollar_treasury - dur_dollar_repo
tab_net.loc['T repo', 'duration'] = net_dur_repo
tab_net.loc['T repo', 'dollar duration'] = net_dur_dollar_repo

# For Swap: Fixed Leg - Floating Leg
net_dur_swap = dur_swap_fix - dur_swap_float
net_dur_dollar_swap = dur_dollar_swap_fix - dur_dollar_swap_float
tab_net.loc['swap', 'duration'] = net_dur_swap
tab_net.loc['swap', 'dollar duration'] = net_dur_dollar_swap

tab_net.loc['net'] = tab_net.loc['T repo'] - tab_net.loc['swap']

tab_net

,duration,dollar duration
T repo,16.583633,1748.425100
swap,16.712744,1671.274445
net,-0.129111,77.150654


* Is the duration for the "paying-fixed" swap positive or negative? Is it bigger or smaller in magnitude than the T-bond?

The duration of the "paying-fixed" swap positive is positive. Duration number is always positive. 

In magnitude, the duration of the swap is bigger than that of the T-bond.

## 1.3

What hedge ratio should be used to balance the notional size of the Treasury bond with the notional size of the swap, such that it is a duration-neutral position?

Specifically, if the trader enters the swap paying fixed on \$500 million notional, how large of a position should they take in the Treasury bond?

In [79]:
# hedge ratio = d$i / d$j
hedge_ratio = (net_dur_dollar_swap) / (net_dur_dollar_repo)
print(f"Hedge Ratio: {hedge_ratio:,.4f}")

Hedge Ratio: 0.9559


In [81]:
swap_notional = 500_000_000
par_value = 100

# nj = ni * d$i / d$j
treasury_notional = swap_notional * hedge_ratio
print(f"Notional for Treasury Bond: ${treasury_notional:,.1f}")

# Number of Units = Notional Size / Par Value
treasury_n = treasury_notional / treasury_price_nov08
print(f"Units of Treasury Bond: {treasury_n:,.1f}")

Notional for Treasury Bond: $477,937,100.6
Units of Treasury Bond: 4,542,554.8


In [82]:
par_value = 100

# Number of Units = Notional Size / Par Value
swap_n = swap_notional / par_value
print(f"Notional for Swap: ${swap_notional:,.1f}")
print(f"Units for Swap: {-swap_n:,.1f}")

Notional for Swap: $500,000,000.0
Units for Swap: -5,000,000.0


## 1.4

Suppose it is May 4, 2009, exactly six months after putting the trade on.

The spread is at -28 bps due to...
* The YTM on a new 30-year bond has risen to 4.36\%
* The swap rate on a new 30-year swap has dropped to 4.08\%

Explain conceptually how this movement impacts the components of the trade.

What happens if spread becomes negative?
1.	Lower Swap Rate: <br>
a.	We are paying fixed rate on the swap<br>
b.	If swap rates decreased, our 30-year fixed rate on the swap is higher than any fixed rate of newly created swaps<br>
c.	Our pay-fixed swap position will decrease in value (Mark-to-Market loss)<br>
d.	We will have a loss in our swap position<br>

2.	Higher YTM on Treasuries: <br>
a.	Yields are inversely related to price<br>
b.	If Treasury yields increase, the value on Treasuries decrease<br>
c.	We are long on Treasuries (holding treasuries)<br>
d.	Our long position will lose value due to the increase in yields <br>
e.	This will be considered another loss<br>

3.	The floating rate on the swap and repo will still cancel each other out since they move together in the same direction


## 1.5

Calculate the value of the position on May 4, 2009, immediately after the first coupon and swap payments and swap reset. 

* Calculate the revised price of the Treasury bond by assuming you can apply the (May 4) 30-year YTM as a discount rate to the 29.5 year bond. (We are just using this for a rough approximation. You know that good pricing would require a discount curve, but let's not get bogged down with that here.)


* Calculate the value of the swap by decomposing it into a fixed-rate bond and a floating-rate bond.
    * The 29.5 year fixed-rate leg is priced using the (May 4) 30-year swap rate as a discount rate.
    * The floating-rate leg is priced at par given that floating-rate notes are par immediately after resets.
    
**Note**

You are being asked to calculate these valuations using the exact formula between price, cashflows, and YTM discount rate. We are not simply approximating with duration, as we already know the position was set up with zero dollar duration.

From the Discussion notebook, we have this formula expressing a bond's price as a function of the coupon, $c$, and the YTM, $y_j$.

$\begin{align*}
P_j(t,T,c) = \sum_{i=1}^{n-1}\frac{100\left(\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T_i-t)}} + \frac{100\left(1+\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T-t)}}
\end{align*}
$

In [52]:
# New Parameters:
ytm_2 = 4.36/100  # Annual yield to maturity of 30Y Treasuries
swap_rate_2 = 4.08/100 # Fixed rate on 30Y swap

In [85]:
# Pricing Treasuries based on YTM
ytm_2 = 4.36/100  # Annual yield to maturity of 30Y Treasuries
coupon_rate = 4.5/100  # Annual coupon rate
face_value = 100  # Face value of $100
time_to_maturity = 29.5  # Years to maturity

treasury_price_may09 = price_treasury_ytm(ytm_2, coupon_rate, face_value, time_to_maturity)
print(f"The revised price of the Treasury bond on May 4, 2009 is: ${treasury_price_may09:.2f}")

print(f"The original price of the Treasury bond on November 2008 is: ${treasury_price_nov08:.2f}")

The revised price of the Treasury bond on May 4, 2009 is: $102.31
The original price of the Treasury bond on November 2008 is: $105.21


In [84]:
# Pricing Fixed Leg of the Swap

# Use the new 30Y swap rate as the discount rate (YTM) for the 29.5Y Swap
swap_rate_2 = 4.08/100 # Fixed rate on 30Y swap
ytm = swap_rate_2

# Use the old 30Y swap rate as the coupon rate for the 29.5Y Swap
swap_rate = 4.256/100 # Fixed rate on 30Y swap
coupon_rate = swap_rate 

face_value = 100  # Face value of $100
time_to_maturity = 29.5  # Years to maturity

fix_leg_price = price_treasury_ytm(ytm, coupon_rate, face_value, time_to_maturity)
print(f"On May 4, 2009:")
print(f"Price of the Fixed Leg of the Swap: ${fix_leg_price:.2f}")

par_value = 100
print(f"Price of the Floating Leg of the Swap: ${par_value:.2f}")

# Swap Price = Fixed Leg Price - Floating Leg Price
swap_price_may09 = fix_leg_price - par_value
print(f"Price of the Swap: ${swap_price_may09:.2f}\n")

swap_price_nov08 = par_value - par_value
print(f"Price of the Swap on Nov 2008: ${swap_price_nov08:.2f}")

On May 4, 2009:
Price of the Fixed Leg of the Swap: $103.00
Price of the Floating Leg of the Swap: $100.00
Price of the Swap: $3.00

Price of the Swap on Nov 2008: $0.00


## 1.6

Accounting for the change in value of the positions, as well as the 6-month cashflows paid on May 4, 
* what is the net profit and loss (pnl) of the position?
* what is the return on the equity capital, considering that there was a 2\% haircut (equity contribution) on the size of the initial treasury bond position.

In [86]:
# Cash Flow Total = Cash Flow per Security x Number of Securities
cashflow_treasury_total = cashflow_treasury * treasury_n
cashflow_swap_total = cashflow_swap_fix * swap_n
cashflow_net = cashflow_treasury_total + cashflow_swap_total

print(f"Cash Flow from Treasury Position: ${cashflow_treasury_total:,.2f}")
print(f"Cash Flow from Swap Position: ${cashflow_swap_total:,.2f}")
print(f"Cash Flow Net: ${cashflow_net:,.2f}")

Cash Flow from Treasury Position: $10,220,748.36
Cash Flow from Swap Position: $-10,640,000.00
Cash Flow Net: $-419,251.64


In [87]:
# Capital gains = (New Price - Old Price) x Number of Securities
capgains_treasury = (treasury_price_may09 - treasury_price_nov08) * treasury_n
capgains_swap = - (swap_price_may09 - swap_price_nov08) * swap_n
capgains_net = capgains_treasury + capgains_swap

print(f"Capital Gains from Treasury Position: ${capgains_treasury:,.2f}")
print(f"Capital Gains from Swap Position: ${capgains_swap:,.2f}")
print(f"Capital Gains Net: ${capgains_net:,.2f}")

Capital Gains from Treasury Position: $-13,181,953.84
Capital Gains from Swap Position: $-15,016,747.03
Capital Gains Net: $-28,198,700.86


In [88]:
# PnL = Cash Flow Total + Capital Gains
pnl_treasury = cashflow_treasury_total + capgains_treasury
pnl_swap = cashflow_swap_total + capgains_swap
pnl_net = pnl_treasury + pnl_swap

print(f"PnL from Treasury Position: ${pnl_treasury:,.2f}")
print(f"PnL from Swap Position: ${pnl_swap:,.2f}")
print(f"PnL Net: ${pnl_net:,.2f}")

PnL from Treasury Position: $-2,961,205.48
PnL from Swap Position: $-25,656,747.03
PnL Net: $-28,617,952.51


In [89]:
# Asset = Old Price x Number of Securities
asset_treasury = treasury_price_nov08 * treasury_n
asset_swap = swap_price_nov08 * swap_n
asset_net = asset_treasury + asset_swap

print(f"Asset - Treasuries: ${asset_treasury:,.2f}")
print(f"Asset - Swap: ${asset_swap:,.2f}")
print(f"Asset - Net: ${asset_net:,.2f}")

Asset - Treasuries: $477,937,100.62
Asset - Swap: $0.00
Asset - Net: $477,937,100.62


In [90]:
# Equity = Asset * Haircut
haircut = 0.02
equity_treasury = asset_treasury * haircut
equity_swap = asset_swap * haircut
equity_net = equity_treasury + equity_swap

print(f"Equity - Treasuries: ${equity_treasury:,.2f}")
print(f"Equity - Swap: ${equity_swap:,.2f}")
print(f"Equity - Net: ${equity_net:,.2f}")

Equity - Treasuries: $9,558,742.01
Equity - Swap: $0.00
Equity - Net: $9,558,742.01


In [91]:
# Return on Equity = (Net PnL / Net asset) x 100
RoE = (pnl_net/equity_net)*100
print(f"Return on Equity: {RoE:,.2f}%")

Return on Equity: -299.39%


***

# 2. Factor Duration

### Data

This problem uses data from,
* `/data/yields.xlsx`
* `/data/treasury_ts_duration_2024-10-31.xlsx`

#### Load Yields

In [61]:
filepath = '../data/yields.xlsx'
yields = pd.read_excel(filepath, sheet_name='yields')
yields.set_index('caldt',inplace=True)

#### Load Prices and Durations of Two Treasuries

In [62]:
QUOTE_DATE = '2024-10-31'
filepath = f'../data/treasury_ts_duration_{QUOTE_DATE}.xlsx'

data = pd.read_excel(filepath,sheet_name='database')
data_info =  data.drop_duplicates(subset='KYTREASNO', keep='first').set_index('KYTREASNO')
data_info[['type','issue date','maturity date','cpn rate']]

,type,issue date,maturity date,cpn rate
KYTREASNO,,,,
207391,note,2019-08-15,2029-08-15,1.625
207392,bond,2019-08-15,2049-08-15,2.250


You will largely focus on the sheets which give the timeseries of prices and durations for each of the two securities, as shown in the following code.

In [63]:
SHEET_PRICE = 'price'
SHEET_DURATION = 'duration'
INDEX_NAME = 'quote date'

price = pd.read_excel(filepath,sheet_name=SHEET_PRICE).set_index(INDEX_NAME)
duration = pd.read_excel(filepath,sheet_name=SHEET_DURATION).set_index(INDEX_NAME)
price.head()

,207391,207392
quote date,,
2019-08-09,98.882812,99.789062
2019-08-12,99.796875,102.554688
2019-08-13,99.281250,101.867188
2019-08-14,100.406250,105.179688
2019-08-15,100.882812,106.234375


### 2.1.

Construct the following yield-curve factors from the `yields` data set:

$\begin{align}
x^{\text{level}}_t =& \frac{1}{N_{\text{yields}}}\sum_{i=1}^{N_{\text{yields}}} y^{(i)}_t\\
x^{\text{slope}}_t =& y^{(30)}_t - y^{(1)}_t\\
x^{\text{curvature}}_t =& -y^{(1)}_t + 2 y^{(10)}_t - y^{(30)}_t
\end{align}$

In [64]:
df_yield = yields.copy()
df_yield.head()

,1,2,5,7,10,20,30
caldt,,,,,,,
1961-06-14,2.935907,3.360687,3.623677,3.768720,3.818819,3.814210,3.815172
1961-06-15,2.932936,3.376460,3.671691,3.804225,3.862987,3.828220,3.826316
1961-06-16,2.929949,3.375670,3.685431,3.804216,3.863282,3.832922,3.830049
1961-06-19,2.920884,3.389970,3.712984,3.824557,3.886205,3.842378,3.837543
1961-06-20,2.952419,3.355796,3.685391,3.809274,3.886506,3.856465,3.845018


In [65]:
# Level = Mean of all yields
df_yield['Level'] = df_yield[[1, 2, 5, 7, 10, 20, 30]].mean(axis=1)
# Slope = 30-year yield - 1-year yield
df_yield['Slope'] = df_yield[30] - df_yield[1]
# Curvature = - 1-year yield + 2 * 10-year yield - 30-year yield
df_yield['Curvature'] = - df_yield[1] + 2*df_yield[10] - df_yield[30]

df_factors = df_yield[['Level', 'Slope', 'Curvature']]
df_factors.head()

,Level,Slope,Curvature
caldt,,,
1961-06-14,3.591027,0.879264,0.886559
1961-06-15,3.614691,0.893380,0.966721
1961-06-16,3.617360,0.900101,0.966566
1961-06-19,3.630646,0.916659,1.013983
1961-06-20,3.627267,0.892599,0.975574


### 2.2

Get the bond prices and durations for the two bonds in the data set referenced above.

#### Align the data

Align the bond pricing data with the yield factor data, so that you have data for both in the intersection of their dates.


#### Estimate the regression

Estimate the regression in the form of day-over-day differences for both bond prices and factors. That is, we are using regression to approximate the factor duration equation,

$\begin{align}
\frac{dP}{P} = \alpha + \beta_L dx_{\text{level}} + \beta_S dx_{\text{slope}} + \beta_C dx_{\text{curvature}} + \epsilon
\end{align}$

Report the betas for each of these factors, for each of the bond prices.

In [66]:
# inner to keep only matching rows
df_price_factors = price.merge(df_factors, left_index=True, right_index=True, how='inner') 
df_price_factors.head()

,207391,207392,Level,Slope,Curvature
2019-08-09,98.882812,99.789062,1.806375,0.416249,-0.621449
2019-08-12,99.796875,102.554688,1.715649,0.348970,-0.638499
2019-08-13,99.281250,101.867188,1.783262,0.281567,-0.669613
2019-08-14,100.406250,105.179688,1.678094,0.198027,-0.696493
2019-08-15,100.882812,106.234375,1.624473,0.217206,-0.690059


In [67]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Compute percentage changes for bond prices (dP/P)
df_price_factors['207391_pct_change'] = df_price_factors[207391].pct_change()
df_price_factors['207392_pct_change'] = df_price_factors[207392].pct_change()

# Compute first differences for factors (dFactor)
df_price_factors['Level_diff'] = df_price_factors['Level'].diff()
df_price_factors['Slope_diff'] = df_price_factors['Slope'].diff()
df_price_factors['Curvature_diff'] = df_price_factors['Curvature'].diff()

# Drop NaN values due to differencing
df_price_factors.dropna(inplace=True)

# Define independent (X) and dependent (Y) variables for regression
X = df_price_factors[['Level_diff', 'Slope_diff', 'Curvature_diff']]
X = sm.add_constant(X)  # Add intercept

# Regression for bond 207391
y_207391 = df_price_factors['207391_pct_change']
model_207391 = sm.OLS(y_207391, X).fit()

# Regression for bond 207392
y_207392 = df_price_factors['207392_pct_change']
model_207392 = sm.OLS(y_207392, X).fit()

# Print results
print("Regression Results for Bond 207391:")
print(model_207391.summary())

Regression Results for Bond 207391:
                            OLS Regression Results                            
Dep. Variable:      207391_pct_change   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.942
Method:                 Least Squares   F-statistic:                     7129.
Date:                Mon, 03 Feb 2025   Prob (F-statistic):               0.00
Time:                        14:47:43   Log-Likelihood:                 7196.5
No. Observations:                1329   AIC:                        -1.439e+04
Df Residuals:                    1325   BIC:                        -1.436e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const   

In [68]:
print("\nRegression Results for Bond 207392:")
print(model_207392.summary())


Regression Results for Bond 207392:
                            OLS Regression Results                            
Dep. Variable:      207392_pct_change   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.960
Method:                 Least Squares   F-statistic:                 1.066e+04
Date:                Mon, 03 Feb 2025   Prob (F-statistic):               0.00
Time:                        14:47:43   Log-Likelihood:                 6148.4
No. Observations:                1329   AIC:                        -1.229e+04
Df Residuals:                    1325   BIC:                        -1.227e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const  

In [69]:
# Print coefficients (alpha and betas) for Bond 207391
print("Regression Parameters for Bond 207391:")
print(model_207391.params)  # Alpha and Betas

# Print coefficients (alpha and betas) for Bond 207392
print("\nRegression Parameters for Bond 207392:")
print(model_207392.params)

Regression Parameters for Bond 207391:
const             0.000064
Level_diff       -0.069938
Slope_diff       -0.004759
Curvature_diff   -0.010926
dtype: float64

Regression Parameters for Bond 207392:
const             0.000069
Level_diff       -0.197983
Slope_diff       -0.126491
Curvature_diff    0.074870
dtype: float64


### 2.3.

Compare the "level" factor beta for each of the two treasuries with the average  duration for each bond as reported in the data set.

* How closely does the average duration for a bond compare to its "level" beta?
* What do you conclude about the usefulness of mathematical duration vs regression sensitivities?

In [70]:
# inner to keep only matching rows
df_dur_factors = duration.merge(df_factors, left_index=True, right_index=True, how='inner') 
df_dur_factors.head(2)

,207391,207392,Level,Slope,Curvature
2019-08-09,9.289497,22.000102,1.806375,0.416249,-0.621449
2019-08-12,9.285468,22.118496,1.715649,0.348970,-0.638499


In [71]:
# Level factor
level_207391 = (model_207391.params.iloc[1])
print(f"Level Factor Beta - 207391: {level_207391:,.4f}")

# Level factor
level_207392 = (model_207392.params.iloc[1])
print(f"Level Factor Beta - 207392: {level_207392:,.4f}")

Level Factor Beta - 207391: -0.0699
Level Factor Beta - 207392: -0.1980


In [72]:
dur_ave_207391 = df_dur_factors[207391].mean()
print(f"Duration - 207391: {dur_ave_207391:,.4f}")

dur_ave_207392 = df_dur_factors[207392].mean()
print(f"Duration - 207392: {dur_ave_207392:,.4f}")

# Level factor
level_207391 = (model_207391.params.iloc[1]) * -100
print(f"Level Factor Beta Adjusted - 207391: {level_207391:,.4f}")

# Level factor
level_207392 = (model_207392.params.iloc[1]) * -100
print(f"Level Factor Beta Adjusted - 207392: {level_207392:,.4f}")

Duration - 207391: 6.9237
Duration - 207392: 19.9032
Level Factor Beta Adjusted - 207391: 6.9938
Level Factor Beta Adjusted - 207392: 19.7983


* How closely does the average duration for a bond compare to its "level" beta?

It seems the average duration is (-100) times the level beta for each bond. Once we multiply the level beta by (-100), the average duration and the adjusted level beta are very close, with the level beta slightly overestimating the duration numbers.

* What do you conclude about the usefulness of mathematical duration vs regression sensitivities?

Both are useful in measuring the sensitivity for how a bond's price will change due to changes in interest rate. The mathematical duration is a direct way of computing duration and the regression sensitivities are an estimation method. However, the regression method overestimates the sensitivity of the bond's price to changes in interest rate. 

### 2.4.

In the duration-hedged trade of `Homework 2, Section 2`, was the that trade was long or short this slope factor? 

Do you think the slope factor exposure had a large impact on the trade?

No new analysis needed, just draw a conclusion from the estimates above along with the trade construction in `HW 2, Sec 2`.

In homework 2, section 2, we are long in security `207391` and short for security `207392`.

For 207391, the coefficient for the slope factor is -0.005585. <br>
For 207392, the coefficient for the slope factor is -0.130103.

This means we are going long on the security with a lower (in magnitude) slope factor and we are going short on the security with a higher (in magnitude) slope factor. In other words, the trade was short the slope factor.

No, the slope factor exposure does not have a large impact on the trade. The level factor has a larger impact on the trade. To illustrate, for 207392, the level factor coeficient is -0.203368, which is bigger (in magnitude), compared to the coefficient for the slope factor. 


We are long the short-dated note and short the long-dated bond. As such, we expect the short-dated yield to come down relative to the long-dated yield. You can reconcile this with our slope factor and see if this implies the slope factor goes up (down), in which case you would be long (short). 

Obviously, it would depend on which sign you used when creating your slope factor (e.g. front yield minus back yield instead of back yield minus front yield).

Given the massive duration and maturity mismatch between our bonds, our exposure to a slope factor would be pretty massive in this trade.

***

# 3 Calculating Duration Via Individual Cashflows

## *Optional, not submitted*

Use the data file `../data/treasury_quotes_2024-10-31.xlsx`.

### 3.1 
Set up the cashflow matrix. 

### 3.2
Extract the Nelson-Siegel spot discount curve, as you did in `Homework 1`.

### 3.3
For each treasury issue, calculate the duration as the weighted average of the (discounted!) cashflow maturity.

Report the summary statistics of the durations. (Use `.describe()` from pandas.)

### 3.4
How close are your duration estimates to the imputed durations given in the data source?

Report the summary statistics of the imputed durations minus your calculated durations from above.

### 3.5
Continue using your assumed discount rates of `4.5`\% to calculate the convexity of each issue.

Report the summary statistics of these convexity calculations.

***